## Feature Engineering and CV based on 5-th Place Solutions

`param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}`
         
`n_repeats=2
n_trees = 150`

Train on the last three months

New in this notebook:
- Create training data for all months (2015-02-28 to 2016-04-28) and validate on 2016-05-28

In [1]:
from santander_helper import *

Load all months' data

In [2]:
x_train_list = []
y_train_list = []
w_train_list = []
fixed_lag = 6
for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
    if m in ['2015-01-28', '2016-06-28']:
        continue
    x_tmp, y_tmp, w_tmp = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
    x_train_list.append(x_tmp)
    y_train_list.append(y_tmp)
    w_train_list.append(w_tmp)
del x_tmp, y_tmp, w_tmp
gc.collect()

528

Prepare the last few months for train and validation

In [3]:
# Prepare for train and validation
x_val = x_train_list[-1]
y_val = y_train_list[-1]
w_val = w_train_list[-1]

x_train = pd.concat(x_train_list[-4:-1], axis=0, ignore_index=True, sort=False)
y_train = pd.concat(y_train_list[-4:-1], axis=0, ignore_index=True, sort=False)
w_train = pd.concat(w_train_list[-4:-1], axis=0, ignore_index=True, sort=False)

gc.collect()

0

In [ ]:
param = {'objective': 'multi:softprob', 
         'eta': 0.1, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'merror',
         'min_child_weight': 10,
         'lambda': 5,
         'max_delta_step': 5,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}

n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 3
n_trees = 100
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
df, clfs, running_time = cv_all_month(param, train, val, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=0, verbose_eval=True)

[0]	train-merror:0.290378	val-merror:0.289544	train-MAP@7:0	val-MAP@7:0.887711
[1]	train-merror:0.28353	val-merror:0.279888	train-MAP@7:0	val-MAP@7:0.89762
[2]	train-merror:0.279146	val-merror:0.276278	train-MAP@7:0	val-MAP@7:0.900547
[3]	train-merror:0.277536	val-merror:0.274877	train-MAP@7:0	val-MAP@7:0.90182
[4]	train-merror:0.276755	val-merror:0.274219	train-MAP@7:0	val-MAP@7:0.902457
[5]	train-merror:0.27557	val-merror:0.274029	train-MAP@7:0	val-MAP@7:0.902932
[6]	train-merror:0.274406	val-merror:0.273393	train-MAP@7:0	val-MAP@7:0.903545
[7]	train-merror:0.273747	val-merror:0.273131	train-MAP@7:0	val-MAP@7:0.903849
[8]	train-merror:0.273323	val-merror:0.272627	train-MAP@7:0	val-MAP@7:0.90427
[9]	train-merror:0.272805	val-merror:0.272751	train-MAP@7:0	val-MAP@7:0.904227
[10]	train-merror:0.272084	val-merror:0.27228	train-MAP@7:0	val-MAP@7:0.90481
[11]	train-merror:0.271625	val-merror:0.271442	train-MAP@7:0	val-MAP@7:0.905328
[12]	train-merror:0.271124	val-merror:0.271043	train-MAP@

Train on 1603-1605

In [ ]:
x_train = pd.concat(x_train_list[-3:], axis=0, ignore_index=True, sort=False)
y_train = pd.concat(y_train_list[-3:], axis=0, ignore_index=True, sort=False)
w_train = pd.concat(w_train_list[-3:], axis=0, ignore_index=True, sort=False)

In [ ]:
n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 1
n_trees = 70
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
# val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
clfs, running_time = cv_all_month(param, train, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=47, verbose_eval=True)

Predict 

In [ ]:
x_test = create_test(month='2016-06-28', max_lag=17, fixed_lag=6, pattern_flag=True)

In [ ]:
simulation_name = 'p3'
y_pred, y_sub = predict_all_month(model_dict=clfs, x_test=x_test, 
    sub_name='eda_4_34_{}.csv.gz'.format(simulation_name), n_features=350, n_trees=n_trees)
save_pickle('eda_4_34_results_{}.pickle'.format(simulation_name), (clfs, running_time, y_pred, y_sub))

In [ ]:
y_pred2 = load_pickle('parameter_tune_eda_4_32_p4.pickle')
y_pred2 = y_pred2[-4]

y_pred3 = load_pickle('eda_4_33_results_p2.pickle')
y_pred3 = y_pred3[2]

In [ ]:
y_pred_final = np.concatenate((y_pred, y_pred2, y_pred3))

In [ ]:
y_pred_final = np.mean(y_pred_final, axis=0)

In [ ]:
y_sub = np.argsort(y_pred_final, axis=1)
y_sub = np.fliplr(y_sub)[:, :7]
# Prepare submission
test_id = x_test.loc[:, 'ncodpers'].values
y_sub = [' '.join([target_cols[k] for k in pred]) for pred in y_sub]
y_sub = pd.DataFrame({'ncodpers': test_id, 'added_products': y_sub})
y_sub.to_csv('eda_4_34_{}+eda_4_33_p2+eda_4_32_p4.csv.gz'.format(simulation_name), compression='gzip', index=False)